In [1]:
import networkx as nx
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram

import numpy as np
from scipy.optimize import minimize

In [2]:
# dij需求i到设施j的成本
d = [[1, 2], [1, 2]]
n = 2   # 两个设施点
m = 2   # 两个需求点
# d = [[1, 2], [3, 4], [5, 6]]
# n = 2   # 两个设施点
# m = 3   # 三个需求点
num_qubits = n + 2 * n * m

# gi设施i的建设成本
g = [2, 1]


In [3]:
penalty = 40
depth = 5
params = np.ones(depth * 2)

In [4]:
GateX = np.array([[0, 1],[1, 0]])
GateY = np.array([[0, -1j],[1j, 0]])
GateZ = np.array([[1, 0],[0, -1]])

In [5]:
# dtype=np.complex128
def add_in_target(num_qubits, target_qubit, gate=np.array([[1, 0],[0, -1]])):
    H = np.eye(2 ** (target_qubit))
    H = np.kron(H, gate)
    H = np.kron(H, np.eye(2 ** (num_qubits - 1 - target_qubit)))
    return H


def generate_Hp(n, m, d, g):
    # 初始化 Hp 矩阵为零矩阵
    # print(num_qubits)
    Hp = np.zeros((2**num_qubits, 2**num_qubits))
    for i in range(m):
        for j in range(n):
            Hp += d[i][j] * (add_in_target(num_qubits, n * (1 + i) + j) - np.eye(2**num_qubits)) / 2
    
    for j in range(n):
        Hp +=  g[j - 1] * (add_in_target(num_qubits, j)- np.eye(2**num_qubits)) / 2

    # for i in range(m):
    #     Ht = np.zeros((2**num_qubits, 2**num_qubits))
    #     for j in range(n):
    #         Ht += (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + i) + j)) / 2
    #     Ht -= np.eye(2**num_qubits)
    #     Hp += -penalty * Ht @ Ht
    
    for i in range(m):
        for j in range(n):
            Ht = (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + i) + j)) / 2 + (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + m + i) + j)) / 2 + (np.eye(2**num_qubits) - add_in_target(num_qubits, j)) / 2
            Hp += -penalty * Ht @ Ht

    return Hp

In [6]:
def gnrt_Hd(n, m):
  mn = m * n
  Hd = np.zeros((2**mn, 2**mn)).astype(np.complex128)
  for i in range(mn):
    j = (i + 1) % mn
    Hd += (add_in_target(mn, i, GateX) @ add_in_target(mn, j, GateX) + add_in_target(mn, i, GateY) @ add_in_target(mn, j, GateY))
  return -Hd


In [7]:
from scipy.linalg import expm
def build_circ(n, m, d, g, params):
  qc = QuantumCircuit(num_qubits)
  beta = params[:depth]
  gamma = params[depth:]
  for i in range(m * n):
    qc.h(i)
  for i in range(m * n, m * n * 2, 2):
    qc.x(i)
  for i in range(m * n * 2, num_qubits):
    qc.h(i)
  for dp in range(depth):
    qc.unitary(expm(-1j * gamma[dp] * generate_Hp(n, m, d, g)), range(num_qubits)) # transpile
    # 
    for i in range(m * n):
      qc.rx(beta[dp], i)
    qc.unitary(expm(-1j * beta[dp] * gnrt_Hd(n, m)), range(m * n , 2 * m * n))
    for i in range(2 * m * n, num_qubits):
      qc.rx(beta[dp], i)
  qc.measure_all()
  return qc

In [8]:
print(build_circ(n, m, d, g, np.full(depth * 2, np.pi/4)))

         ┌───┐┌──────────┐┌─────────┐ ┌──────────┐┌─────────┐ ┌──────────┐»
    q_0: ┤ H ├┤0         ├┤ Rx(π/4) ├─┤0         ├┤ Rx(π/4) ├─┤0         ├»
         ├───┤│          │├─────────┤ │          │├─────────┤ │          │»
    q_1: ┤ H ├┤1         ├┤ Rx(π/4) ├─┤1         ├┤ Rx(π/4) ├─┤1         ├»
         ├───┤│          │├─────────┤ │          │├─────────┤ │          │»
    q_2: ┤ H ├┤2         ├┤ Rx(π/4) ├─┤2         ├┤ Rx(π/4) ├─┤2         ├»
         ├───┤│          │├─────────┤ │          │├─────────┤ │          │»
    q_3: ┤ H ├┤3         ├┤ Rx(π/4) ├─┤3         ├┤ Rx(π/4) ├─┤3         ├»
         ├───┤│          │├─────────┴┐│          │├─────────┴┐│          │»
    q_4: ┤ X ├┤4         ├┤0         ├┤4         ├┤0         ├┤4         ├»
         └───┘│  Unitary ││          ││  Unitary ││          ││  Unitary │»
    q_5: ─────┤5         ├┤1         ├┤5         ├┤1         ├┤5         ├»
         ┌───┐│          ││  Unitary ││          ││  Unitary ││          │»
    q_6: ┤ X

In [9]:
def cost_function(x):
  num = [int(char) for char in x]
  C = 0
  for i in range(m):
    for j in range(n):
      C += d[i][j] * num[n * (1 + i) + j]
      
  for j in range(n):
    C += g[j] * num[j]

  # for i in range(m):
  #   t = 0
  #   for j in range(n):
  #     t += num[n * (1 + i) + j]
  #   C += penalty * (t - 1)**2

  for i in range(m):
    for j in range(n):
      C += penalty * (num[n * (1 + i) + j] + num[n * (1 + m + i) + j] - num[j]) ** 2
  return C

def compute_expectation(counts):
  EV = 0
  total_count = 0
  for x, count in counts.items():
    C = cost_function(x)
    EV += C*count
    total_count += count

  return EV/total_count


def expectation_from_sample(shots = 2000):
  backend = Aer.get_backend('qasm_simulator')
  backend.shots = shots

  def execute_circ(theta):
    qc = build_circ(n, m, d, g, theta)
    counts = backend.run(qc, seed_simulator=10, shots=shots).result().get_counts()
    return compute_expectation(counts)
  
  return execute_circ

In [10]:
cost_function('0000010101')

202

In [11]:
from numpy.lib.utils import source
from scipy.optimize import minimize
import numpy as np
# 初始化迭代计数器
iteration_count = 0
def test(pen, dep, par):
  global penalty, depth, params, iteration_count
  iteration_count = 0
  penalty = pen
  depth = dep
  params = par
  expectation = expectation_from_sample()
  def callback(x):
      global iteration_count
      iteration_count += 1
      if iteration_count % 10 == 0:
          print(f"Iteration {iteration_count}, Result: {expectation(x)}")
  # 设定最大迭代次数
  max_iterations = 1000

  # 使用 COBYLA 方法进行最小化，并设置 callback 函数
  res = minimize(expectation, params, method='COBYLA', options={'maxiter': max_iterations}, callback=callback)
  # 输出最终结果
  print("Final Result:", res)
  backend = Aer.get_backend('aer_simulator')
  backend.shots = 100000

  shots=100000
  qc_res = build_circ(n, m, d, g, params=res.x)

  counts = backend.run(qc_res, seed_simulator=10, shots = shots).result().get_counts()
  # plot_histogram(counts)
  sorted_counts = sorted(counts, key=counts.get, reverse=True)
  print("\n----------------- Full result ---------------------")
  print("selection\t\tprobability\tvalue")
  print("---------------------------------------------------")
  for x in sorted_counts[:20]:
    print(x, "{:.1f}%".format(counts[x] / shots * 100), cost_function(x))

In [12]:
test(40, 5, np.full(5 * 2, np.pi/4))

Iteration 10, Result: 138.5655
Iteration 20, Result: 121.236
Iteration 30, Result: 143.0855
Iteration 40, Result: 155.5475
Iteration 50, Result: 96.8595
Iteration 60, Result: 79.654
Iteration 70, Result: 82.798
Iteration 80, Result: 81.152
Iteration 90, Result: 76.334
Iteration 100, Result: 77.941
Iteration 110, Result: 75.6595
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 76.3765
       x: [ 7.723e-01  7.912e-01  1.800e+00  8.011e-01  7.751e-01
            7.740e-01  7.850e-01  1.778e+00  1.730e+00  7.869e-01]
    nfev: 116
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
1010100000 15.2% 4
0010100000 5.1% 82
1010100100 4.8% 44
1010100001 4.6% 44
1101010000 4.0% 87
1000110000 3.7% 85
1001100000 3.7% 85
1011000000 3.6% 85
1010010000 3.6% 85
1110100000 3.4% 85
1010100010 3.0% 44
1010101000 2.9% 44
1001010000 2.3% 166
0101010000 1.4% 5
10

In [13]:
test(40, 3, np.full(3 * 2, np.pi))

Iteration 10, Result: 148.7775
Iteration 20, Result: 163.2095
Iteration 30, Result: 111.9315
Iteration 40, Result: 96.52
Iteration 50, Result: 96.193
Iteration 60, Result: 95.8405
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 94.2535
       x: [ 3.912e+00  2.918e+00  3.144e+00  4.142e+00  3.393e+00
            4.121e+00]
    nfev: 69
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
1101011010 7.7% 7
1010101010 4.3% 84
1101011110 3.7% 47
1101011011 3.7% 47
1101011111 1.8% 87
1100111110 1.5% 46
1111001011 1.5% 46
1101101011 1.5% 46
1001011111 1.4% 326
1110011110 1.4% 46
0001011000 1.3% 124
1101011000 1.3% 47
1101010010 1.3% 47
0001010010 1.3% 124
1010011010 1.1% 85
1110101111 1.1% 85
0101010000 1.1% 5
1000111010 1.1% 85
1001101010 1.1% 85
1001011010 1.1% 86


In [14]:
# for i in range(1, 20):
#   print(f'depth == {i}')
#   test(20, i, np.ones(2 * i))